In [7]:
# Import Python libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Importing the models
from models.mlp import MLP
from models.transformer import Transformer
from models.ft_transformer_wrapper import FTTransformerWrapper
from models.tab_transformer_wrapper import TabTransformerWrapper

In [8]:
def find_best_model_state(model_name, X, y, input_dim, num_classes, stratified_fold, batch_size=64, num_epochs=200, n_folds=10, device='cuda', metric='f1_score', cat_dims=[0, 12, 15, 29, 30], cat_idxs=[12, 3, 3, 3, 3], num_idxs=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]):
    """Get best model out of Cross-Validation, using the specified metric."""
    best_model_state = None
    best_metric = -np.inf
    fold_results_list = []
    for fold, (train_ids, val_ids) in enumerate(stratified_fold.split(X, y)):
        print(f'Fold {fold + 1}/{n_folds}')
        # Prepare data loaders
        train_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(X[train_ids], y[train_ids]),
            batch_size=batch_size, num_workers=0, shuffle=True
        )
        val_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(X[val_ids], y[val_ids]),
            batch_size=batch_size, num_workers=0, 
        )
        # Initialize model, criterion, and optimizer
        if model_name == 'MLP':
            model = MLP(input_dim=input_dim, num_classes=num_classes).to(device)
        elif model_name == 'TabTransformer':
            model = TabTransformerWrapper(cat_dims=cat_dims,cat_idxs=cat_idxs,num_idxs=num_idxs,num_classes=num_classes,num_heads=4,num_layers=2,dim_model=64,dropout=0.1).to(device)
        elif model_name == 'FTTransformer':
            model = FTTransformerWrapper(cat_dims=cat_dims,cat_idxs=cat_idxs,num_idxs=num_idxs,num_classes=num_classes,num_heads=4,num_layers=2,dim_model=64,dim_ff=128,dropout=0.1).to(device)
        elif model_name == 'Transformer':
            model = Transformer(input_dim=input_dim, num_classes=num_classes, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
        else:
            raise ValueError(f"Unknown model name: {model_name}")

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Train model
        model_state = train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=num_epochs)
        model.load_state_dict(model_state)
        # Evaluate model
        result_df = evaluate_model(model, val_loader, device)
        metric_result = result_df[metric].values[0]

        fold_result = {
            'Fold': fold + 1,
            'Model Name': model_name,
        }
        fold_result.update(result_df.to_dict(orient='records')[0])
        fold_results_list.append(fold_result)

        if metric_result > best_metric:
            best_metric = metric_result
            best_model_state = model.state_dict()
        print(f'Best {metric} for fold {fold + 1}: {best_metric:.4f}')
        
    return best_model_state, fold_results_list

def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=500, patience=10):
    best_val_loss = float('inf')
    no_improve = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

    return best_model_state

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    all_attentions = []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.numpy())
            all_probs.extend(probs.cpu().numpy())

            # Get attention weights
            if hasattr(model, 'get_attention_weights') and callable(model.get_attention_weights):
                try:
                    attention_weights = model.get_attention_weights(batch_x)
                    if attention_weights is not None:  # Make sure it's not None
                        all_attentions.append(attention_weights.cpu().numpy())
                except Exception as e:
                    print(f"Error getting attention weights: {e}")
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
    
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'attention_weights': None # np.concatenate(all_attentions, axis=0) if len(all_attentions) > 0 else None
    }
    
    return pd.DataFrame([results])

def save_model_state(model_state, output_path):
    torch.save(model_state, output_path)
    print(f'Model state saved to {output_path}')

In [9]:
# System settings
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# TabTransformer Model Settings

# Load data
train_df = pd.read_csv('./data/3_train_processed.csv')
train_ros_df = pd.read_csv('./data/3_train_ros_processed.csv')
train_smote_df = pd.read_csv('./data/3_train_smote_processed.csv')
test_df = pd.read_csv('./data/3_test_processed.csv')

# Preprocess data
train_features = train_df.drop(['credit_score'], axis=1)
train_ros_features = train_ros_df.drop(['credit_score'], axis=1)
train_smote_features = train_smote_df.drop(['credit_score'], axis=1)
test_features = test_df.drop(['credit_score'], axis=1)

train_labels = train_df['credit_score']
train_ros_labels = train_ros_df['credit_score']
train_smote_labels = train_smote_df['credit_score']
test_labels = test_df['credit_score']

X = torch.FloatTensor(train_features.values)
X_ros = torch.FloatTensor(train_ros_features.values)
X_smote = torch.FloatTensor(train_smote_features.values)
X_test = torch.FloatTensor(test_features.values)
y = torch.LongTensor(train_labels.values)
y_ros = torch.LongTensor(train_ros_labels.values)
y_smote = torch.LongTensor(train_smote_labels.values)
y_test = torch.LongTensor(test_labels.values)

# Transformer Model Settings
# 1. Categorical features
cat_columns = ['month', 'credit_mix', 'payment_of_min_amount', 'spending_level', 'payment_size']
cat_idxs = [0, 12, 15, 29, 30]
# 2. Continuous features - Month could have 12 unique values, so we treat it as a categorical feature
cat_dims = [12, train_df['credit_mix'].nunique(), train_df['payment_of_min_amount'].nunique(), 
            train_df['spending_level'].nunique() + 1, train_df['payment_size'].nunique()]
# 3. Other columns
all_columns = list(train_df.columns)
num_idxs = [i for i in range(len(all_columns)) if i not in cat_idxs and i != all_columns.index('credit_score')]  # 排除目标变量

In [10]:
# fold
n_folds = 10
batch_size = 128
num_epochs = 500
# MLP
stratified_fold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_ros = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_smote = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print('MLP - original data')
mlp_best_state, mlp_fold_results_list = find_best_model_state('MLP', X, y, input_dim=X.shape[1], num_classes=3, stratified_fold=stratified_fold, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
print('MLP - ROS data')
mlp_best_ros_state, mlp_fold_results_list_ros = find_best_model_state('MLP', X_ros, y_ros, input_dim=X_ros.shape[1], num_classes=3, stratified_fold=stratified_fold_ros, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
print('MLP - SMOTE data')
mlp_best_smote_state, mlp_fold_results_list_smote = find_best_model_state('MLP', X_smote, y_smote, input_dim=X_smote.shape[1], num_classes=3, stratified_fold=stratified_fold_smote, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
save_model_state(mlp_best_state, './models/mlp_best_state.pth')
save_model_state(mlp_best_ros_state, './models/mlp_best_ros_state.pth')
save_model_state(mlp_best_smote_state, './models/mlp_best_smote_state.pth')
# Save csv
mlp_fold_results_df = pd.DataFrame(mlp_fold_results_list).to_csv('./models/mlp_fold_results.csv', index=False)
mlp_fold_results_df_ros = pd.DataFrame(mlp_fold_results_list_ros).to_csv('./models/mlp_fold_results_ros.csv', index=False)
mlp_fold_results_df_smote = pd.DataFrame(mlp_fold_results_list_smote).to_csv('./models/mlp_fold_results_smote.csv', index=False)


MLP - original data
Fold 1/10
Epoch [1/500], Train Loss: 0.7424, Val Loss: 0.6865
Epoch [10/500], Train Loss: 0.6165, Val Loss: 0.6282
Epoch [20/500], Train Loss: 0.5765, Val Loss: 0.6123
Epoch [30/500], Train Loss: 0.5449, Val Loss: 0.5923
Epoch [40/500], Train Loss: 0.5255, Val Loss: 0.5779
Epoch [50/500], Train Loss: 0.5070, Val Loss: 0.5667
Epoch [60/500], Train Loss: 0.4945, Val Loss: 0.5555
Epoch [70/500], Train Loss: 0.4827, Val Loss: 0.5584
Epoch [80/500], Train Loss: 0.4723, Val Loss: 0.5476
Epoch [90/500], Train Loss: 0.4695, Val Loss: 0.5394
Epoch [100/500], Train Loss: 0.4628, Val Loss: 0.5429
Epoch [110/500], Train Loss: 0.4570, Val Loss: 0.5402
Early stopping at epoch 111
Best f1_score for fold 1: 0.7578
Fold 2/10
Epoch [1/500], Train Loss: 0.7440, Val Loss: 0.6893
Epoch [10/500], Train Loss: 0.6168, Val Loss: 0.6314
Epoch [20/500], Train Loss: 0.5735, Val Loss: 0.6141
Epoch [30/500], Train Loss: 0.5424, Val Loss: 0.5998
Epoch [40/500], Train Loss: 0.5198, Val Loss: 0.584

In [11]:
# fold
n_folds = 10
batch_size = 128
num_epochs = 500
# Transformer
stratified_fold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_ros = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_smote = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
print('Transformer - original data')
transformer_best_state, transformer_fold_results_list = find_best_model_state('Transformer', X, y, input_dim=X.shape[1], num_classes=3, stratified_fold=stratified_fold, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
print('Transformer - ROS data')
transformer_best_ros_state, transformer_fold_results_list_ros = find_best_model_state('Transformer', X_ros, y_ros, input_dim=X_ros.shape[1], num_classes=3, stratified_fold=stratified_fold_ros, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
print('Transformer - SMOTE data')
transformer_best_smote_state, transformer_fold_results_list_smote = find_best_model_state('Transformer', X_smote, y_smote, input_dim=X_smote.shape[1], num_classes=3, stratified_fold=stratified_fold_smote, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device)
save_model_state(transformer_best_state, './models/transformer_best_state.pth')
save_model_state(transformer_best_ros_state, './models/transformer_best_ros_state.pth')
save_model_state(transformer_best_smote_state, './models/transformer_best_smote_state.pth')
# Save csv
transformer_fold_results_df = pd.DataFrame(transformer_fold_results_list).to_csv('./models/transformer_fold_results.csv', index=False)
transformer_fold_results_df_ros = pd.DataFrame(transformer_fold_results_list_ros).to_csv('./models/transformer_fold_results_ros.csv', index=False)
transformer_fold_results_df_smote = pd.DataFrame(transformer_fold_results_list_smote).to_csv('./models/transformer_fold_results_smote.csv', index=False)

Transformer - original data
Fold 1/10
Epoch [1/500], Train Loss: 0.7123, Val Loss: 0.6636
Epoch [10/500], Train Loss: 0.6245, Val Loss: 0.6305
Epoch [20/500], Train Loss: 0.5775, Val Loss: 0.6051
Epoch [30/500], Train Loss: 0.5038, Val Loss: 0.5553
Epoch [40/500], Train Loss: 0.4440, Val Loss: 0.5200
Epoch [50/500], Train Loss: 0.3999, Val Loss: 0.5144
Epoch [60/500], Train Loss: 0.3643, Val Loss: 0.5098
Early stopping at epoch 61
Best f1_score for fold 1: 0.7963
Fold 2/10
Epoch [1/500], Train Loss: 0.7163, Val Loss: 0.6601
Epoch [10/500], Train Loss: 0.6266, Val Loss: 0.6287
Epoch [20/500], Train Loss: 0.5873, Val Loss: 0.6064
Epoch [30/500], Train Loss: 0.5327, Val Loss: 0.5768
Epoch [40/500], Train Loss: 0.4707, Val Loss: 0.5597
Epoch [50/500], Train Loss: 0.4259, Val Loss: 0.5209
Epoch [60/500], Train Loss: 0.3924, Val Loss: 0.5167
Early stopping at epoch 63
Best f1_score for fold 2: 0.7963
Fold 3/10
Epoch [1/500], Train Loss: 0.7145, Val Loss: 0.6797
Epoch [10/500], Train Loss: 0.

In [12]:
# fold
n_folds = 10
batch_size = 128
num_epochs = 500
# TabTransformer
stratified_fold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_ros = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_smote = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print('TabTransformer - original data')
tab_transformer_best_state, tab_transformer_fold_results_list = find_best_model_state('TabTransformer', X, y, input_dim=X.shape[1], num_classes=3, stratified_fold=stratified_fold, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
tab_transformer_best_ros_state, tab_transformer_fold_results_list_ros = find_best_model_state('TabTransformer', X_ros, y_ros, input_dim=X_ros.shape[1], num_classes=3, stratified_fold=stratified_fold_ros, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
tab_transformer_best_smote_state, tab_transformer_fold_results_list_smote = find_best_model_state('TabTransformer', X_smote, y_smote, input_dim=X_smote.shape[1], num_classes=3, stratified_fold=stratified_fold_smote, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
save_model_state(tab_transformer_best_state, './models/tab_transformer_best_state.pth')
save_model_state(tab_transformer_best_ros_state, './models/tab_transformer_best_ros_state.pth')
save_model_state(tab_transformer_best_smote_state, './models/tab_transformer_best_smote_state.pth')
# Save csv
tab_transformer_fold_results_df = pd.DataFrame(tab_transformer_fold_results_list).to_csv('./models/tab_transformer_fold_results.csv', index=False)
tab_transformer_fold_results_df_ros = pd.DataFrame(tab_transformer_fold_results_list_ros).to_csv('./models/tab_transformer_fold_results_ros.csv', index=False)
tab_transformer_fold_results_df_smote = pd.DataFrame(tab_transformer_fold_results_list_smote).to_csv('./models/tab_transformer_fold_results_smote.csv', index=False)

TabTransformer - original data
Fold 1/10
Epoch [1/500], Train Loss: 0.7177, Val Loss: 0.6738
Epoch [10/500], Train Loss: 0.5999, Val Loss: 0.6273
Epoch [20/500], Train Loss: 0.2390, Val Loss: 0.7052
Early stopping at epoch 24
Best f1_score for fold 1: 0.7346
Fold 2/10
Epoch [1/500], Train Loss: 0.7190, Val Loss: 0.6679
Epoch [10/500], Train Loss: 0.5941, Val Loss: 0.6275
Epoch [20/500], Train Loss: 0.2279, Val Loss: 0.7234
Early stopping at epoch 24
Best f1_score for fold 2: 0.7492
Fold 3/10
Epoch [1/500], Train Loss: 0.7197, Val Loss: 0.6839
Epoch [10/500], Train Loss: 0.6041, Val Loss: 0.6322
Epoch [20/500], Train Loss: 0.2793, Val Loss: 0.7147
Early stopping at epoch 24
Best f1_score for fold 3: 0.7492
Fold 4/10
Epoch [1/500], Train Loss: 0.7250, Val Loss: 0.6813
Epoch [10/500], Train Loss: 0.5999, Val Loss: 0.6242
Epoch [20/500], Train Loss: 0.2517, Val Loss: 0.7397
Early stopping at epoch 24
Best f1_score for fold 4: 0.7492
Fold 5/10
Epoch [1/500], Train Loss: 0.7248, Val Loss: 0.

In [ ]:
# fold
n_folds = 10
batch_size = 128
num_epochs = 500
# FTTransformer
stratified_fold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_ros = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
stratified_fold_smote = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print('FTTransformer - original data')
ft_transformer_best_state, ft_transformer_fold_results_list = find_best_model_state('FTTransformer', X, y, input_dim=X.shape[1], num_classes=3, stratified_fold=stratified_fold, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
print('FTTransformer - ROS data')
ft_transformer_best_ros_state, ft_transformer_fold_results_list_ros = find_best_model_state('FTTransformer', X_ros, y_ros, input_dim=X_ros.shape[1], num_classes=3, stratified_fold=stratified_fold_ros, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
print('FTTransformer - SMOTE data')
ft_transformer_best_smote_state, ft_transformer_fold_results_list_smote = find_best_model_state('FTTransformer', X_smote, y_smote, input_dim=X_smote.shape[1], num_classes=3, stratified_fold=stratified_fold_smote, batch_size=batch_size, num_epochs=num_epochs, n_folds=n_folds, device=device, cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs)
save_model_state(ft_transformer_best_state, './models/ft_transformer_best_state.pth')
save_model_state(ft_transformer_best_ros_state, './models/ft_transformer_best_ros_state.pth')
save_model_state(ft_transformer_best_smote_state, './models/ft_transformer_best_smote_state.pth')
# Save csv
ft_transformer_fold_results_df = pd.DataFrame(ft_transformer_fold_results_list).to_csv('./models/ft_transformer_fold_results.csv', index=False)
ft_transformer_fold_results_df_ros = pd.DataFrame(ft_transformer_fold_results_list_ros).to_csv('./models/ft_transformer_fold_results_ros.csv', index=False)
ft_transformer_fold_results_df_smote = pd.DataFrame(ft_transformer_fold_results_list_smote).to_csv('./models/ft_transformer_fold_results_smote.csv', index=False)

FTTransformer - original data
Fold 1/10
Epoch [1/500], Train Loss: 0.7190, Val Loss: 0.6755
Epoch [10/500], Train Loss: 0.6264, Val Loss: 0.6249
Epoch [20/500], Train Loss: 0.5990, Val Loss: 0.6119
Epoch [30/500], Train Loss: 0.5581, Val Loss: 0.5866
Epoch [40/500], Train Loss: 0.5139, Val Loss: 0.5537
Epoch [50/500], Train Loss: 0.4755, Val Loss: 0.5247
Epoch [60/500], Train Loss: 0.4429, Val Loss: 0.5116
Epoch [70/500], Train Loss: 0.4179, Val Loss: 0.4994
Epoch [80/500], Train Loss: 0.4009, Val Loss: 0.4984
Epoch [90/500], Train Loss: 0.3848, Val Loss: 0.4794
Early stopping at epoch 99
Best f1_score for fold 1: 0.8098
Fold 2/10
Epoch [1/500], Train Loss: 0.7207, Val Loss: 0.6832
Epoch [10/500], Train Loss: 0.6259, Val Loss: 0.6269
Epoch [20/500], Train Loss: 0.5914, Val Loss: 0.5998
Epoch [30/500], Train Loss: 0.5406, Val Loss: 0.5575
Epoch [40/500], Train Loss: 0.4913, Val Loss: 0.5257
Epoch [50/500], Train Loss: 0.4552, Val Loss: 0.5118
Epoch [60/500], Train Loss: 0.4232, Val Loss

In [ ]:
final_fold_results_df = pd.concat([pd.DataFrame(mlp_fold_results_list), pd.DataFrame(mlp_fold_results_list_ros), pd.DataFrame(mlp_fold_results_list_smote), pd.DataFrame(transformer_fold_results_list), pd.DataFrame(transformer_fold_results_list_ros), pd.DataFrame(transformer_fold_results_list_smote), pd.DataFrame(tab_transformer_fold_results_list), pd.DataFrame(tab_transformer_fold_results_list_ros), pd.DataFrame(tab_transformer_fold_results_list_smote), pd.DataFrame(ft_transformer_fold_results_list), pd.DataFrame(ft_transformer_fold_results_list_ros), pd.DataFrame(ft_transformer_fold_results_list_smote)], ignore_index=True)
final_fold_results_df.to_csv('./data/5_final_fold_results.csv', index=False)
final_fold_results_df

,Fold,Model Name,accuracy,precision,recall,f1_score,roc_auc,attention_weights
0,1,TabTransformer,0.809001,0.794442,0.808112,0.800580,0.924369,"[[[0.006546152, 0.009071659, 0.0020918595, 0.0..."
1,2,TabTransformer,0.808298,0.796923,0.809708,0.802561,0.925042,"[[[0.010708559, 0.015709331, 0.029624566, 0.02..."
2,3,TabTransformer,0.786639,0.768705,0.788117,0.777193,0.915547,"[[[8.279453e-05, 0.073742904, 0.00079792726, 0..."
3,4,TabTransformer,0.808158,0.793229,0.810284,0.800564,0.923616,"[[[0.004616289, 0.004747437, 0.013187103, 0.00..."
4,5,TabTransformer,0.794796,0.778552,0.796974,0.786246,0.916319,"[[[0.0021005424, 0.0046139136, 0.02102275, 0.0..."
5,6,TabTransformer,0.804641,0.794015,0.795723,0.794534,0.922728,"[[[0.00026688754, 0.0003841871, 0.00022494567,..."
6,7,TabTransformer,0.796906,0.777422,0.797173,0.786080,0.919494,"[[[0.0043111523, 0.006890977, 0.00679004, 0.00..."
7,8,TabTransformer,0.785765,0.766161,0.788108,0.775889,0.917160,"[[[0.16144754, 0.0061347005, 0.0021467428, 0.0..."
8,9,TabTransformer,0.806302,0.790891,0.810157,0.799399,0.921811,"[[[0.0705701, 0.018688142, 0.023701102, 0.0021..."
9,10,TabTransformer,0.805599,0.795173,0.799345,0.797182,0.925338,"[[[0.0017049853, 0.01803071, 0.011090298, 0.00..."


In [ ]:
# Load best model
mlp_best_model_state = torch.load('./models/mlp_best_state.pth')
mlp_best_model = MLP(input_dim=X.shape[1], num_classes=3).to(device)
mlp_best_model.load_state_dict(mlp_best_model_state)
mlp_best_roc_model_state = torch.load('./models/mlp_best_ros_state.pth')
mlp_best_roc_model = MLP(input_dim=X.shape[1], num_classes=3).to(device)
mlp_best_roc_model.load_state_dict(mlp_best_roc_model_state)
mlp_best_smote_model_state = torch.load('./models/mlp_best_smote_state.pth')
mlp_best_smote_model = MLP(input_dim=X.shape[1], num_classes=3).to(device)
mlp_best_smote_model.load_state_dict(mlp_best_smote_model_state)
transformer_best_model_state = torch.load('./models/transformer_best_state.pth')
transformer_best_model = Transformer(input_dim=X.shape[1], num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
transformer_best_model.load_state_dict(transformer_best_model_state)
transformer_best_roc_model_state = torch.load('./models/transformer_best_ros_state.pth')
transformer_best_roc_model = Transformer(input_dim=X.shape[1], num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
transformer_best_roc_model.load_state_dict(transformer_best_roc_model_state)
transformer_best_smote_model_state = torch.load('./models/transformer_best_smote_state.pth')
transformer_best_smote_model = Transformer(input_dim=X.shape[1], num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
transformer_best_smote_model.load_state_dict(transformer_best_smote_model_state)
ft_transformer_best_model_state = torch.load('./models/ft_transformer_best_state.pth')
ft_transformer_best_model = FTTransformerWrapper(cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs, num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
ft_transformer_best_model.load_state_dict(ft_transformer_best_model_state)
ft_transformer_best_roc_model_state = torch.load('./models/ft_transformer_best_ros_state.pth')
ft_transformer_best_roc_model = FTTransformerWrapper(cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs, num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
ft_transformer_best_roc_model.load_state_dict(ft_transformer_best_roc_model_state)
ft_transformer_best_smote_model_state = torch.load('./models/ft_transformer_best_smote_state.pth')
ft_transformer_best_smote_model = FTTransformerWrapper(cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs, num_classes=3, num_heads=4, num_layers=2, dim_model=64, dim_ff=128, dropout=0.1).to(device)
ft_transformer_best_smote_model.load_state_dict(ft_transformer_best_smote_model_state)
tab_transformer_best_model_state = torch.load('./models/tab_transformer_best_state.pth')
tab_transformer_best_model = TabTransformerWrapper(cat_dims=cat_dims,cat_idxs=cat_idxs,num_idxs=num_idxs,num_classes=3,num_heads=4,num_layers=2, dim_model=64, dropout=0.1).to(device)
tab_transformer_best_model.load_state_dict(tab_transformer_best_model_state)
tab_transformer_best_roc_model_state = torch.load('./models/tab_transformer_best_ros_state.pth')
tab_transformer_best_roc_model = TabTransformerWrapper(cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs, num_classes=3, num_heads=4, num_layers=2, dim_model=64, dropout=0.1).to(device)
tab_transformer_best_roc_model.load_state_dict(tab_transformer_best_roc_model_state)
tab_transformer_best_smote_model_state = torch.load('./models/tab_transformer_best_smote_state.pth')
tab_transformer_best_smote_model = TabTransformerWrapper(cat_dims=cat_dims, cat_idxs=cat_idxs, num_idxs=num_idxs, num_classes=3, num_heads=4, num_layers=2, dim_model=64, dropout=0.1).to(device)
tab_transformer_best_smote_model.load_state_dict(tab_transformer_best_smote_model_state)

def evaluate_model(model, X_test, y_test, device, name='MLP'):
    """Evaluate the model on the test set."""
    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_test, y_test),
        batch_size=128, num_workers=0
    )
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

    results = {
        'name': name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }

    return pd.DataFrame([results])

# Evaluate best model
result_mlp_df = evaluate_model(mlp_best_model, X_test, y_test, device, name='MLP')
result_mlp_roc_df = evaluate_model(mlp_best_roc_model, X_test, y_test, device, name='MLP_ROS')
result_mlp_smote_df = evaluate_model(mlp_best_smote_model, X_test, y_test, device, name='MLP_SMOTE')
result_transformer_df = evaluate_model(transformer_best_model, X_test, y_test, device, name='Transformer')
result_transformer_roc_df = evaluate_model(transformer_best_roc_model, X_test, y_test, device, name='Transformer_ROS')
result_transformer_smote_df = evaluate_model(transformer_best_smote_model, X_test, y_test, device, name='Transformer_SMOTE')
result_ft_transformer_df = evaluate_model(ft_transformer_best_model, X_test, y_test, device, name='FTTransformer')
result_ft_transformer_roc_df = evaluate_model(ft_transformer_best_roc_model, X_test, y_test, device, name='FTTransformer_ROS')
result_ft_transformer_smote_df = evaluate_model(ft_transformer_best_smote_model, X_test, y_test, device, name='FTTransformer_SMOTE')
result_tab_transformer_df = evaluate_model(tab_transformer_best_model, X_test, y_test, device, name='TabTransformer')
result_tab_transformer_roc_df = evaluate_model(tab_transformer_best_roc_model, X_test, y_test, device, name='TabTransformer_ROS')
result_tab_transformer_smote_df = evaluate_model(tab_transformer_best_smote_model, X_test, y_test, device, name='TabTransformer_SMOTE')

# Combine results
result_df = pd.concat([
    result_mlp_df, 
    result_mlp_roc_df, 
    result_mlp_smote_df, 
    result_transformer_df, 
    result_transformer_roc_df, 
    result_transformer_smote_df,
    result_tab_transformer_df,
    result_tab_transformer_roc_df,
    result_tab_transformer_smote_df,
    result_ft_transformer_df,
    result_ft_transformer_roc_df,
    result_ft_transformer_smote_df
], ignore_index=True)
result_df.to_csv('./data/5_test_result.csv', index=False)

RuntimeError: Error(s) in loading state_dict for TabTransformerWrapper:
	size mismatch for model.mlp.mlp.0.weight: copying a param with shape torch.Size([722, 361]) from checkpoint, the shape in current model is torch.Size([1444, 361]).
	size mismatch for model.mlp.mlp.0.bias: copying a param with shape torch.Size([722]) from checkpoint, the shape in current model is torch.Size([1444]).
	size mismatch for model.mlp.mlp.2.weight: copying a param with shape torch.Size([361, 722]) from checkpoint, the shape in current model is torch.Size([722, 1444]).
	size mismatch for model.mlp.mlp.2.bias: copying a param with shape torch.Size([361]) from checkpoint, the shape in current model is torch.Size([722]).
	size mismatch for model.mlp.mlp.4.weight: copying a param with shape torch.Size([3, 361]) from checkpoint, the shape in current model is torch.Size([3, 722]).

In [ ]:
result_df

,name,accuracy,precision,recall,f1_score,roc_auc
0,MLP,0.768270,0.751040,0.769231,0.758920,0.903942
0,MLP_ROS,0.761744,0.744606,0.803901,0.760298,0.906396
0,MLP_SMOTE,0.759100,0.738338,0.768856,0.750945,0.897471
0,TabTransformer,0.784473,0.775707,0.777907,0.775074,0.913639
0,TabTransformer_ROS,0.793249,0.777044,0.828348,0.792344,0.917298
0,TabTransformer_SMOTE,0.791786,0.777341,0.807728,0.788842,0.915378
